In [1]:
import json
import csv
import pandas as pd
import nltk
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, RegexpTokenizer
#import tensorflow as tf
import numpy as np
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to C:\Users\CCC-
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\CCC-
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Dense, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


In [3]:

file1 = r'C:\Users\CCC-PC\Desktop\Amazon Book\review_dataset.csv'


df1 = pd.read_csv(file1, on_bad_lines='skip', low_memory=False)


In [4]:
df1.head()

,overall,reviewText,review_label
0,2.0,the title says it all ive loved the authors ot...,0
1,2.0,the book has good principles but does not help...,0
2,1.0,i dont understand why buy this version by jane...,0
3,2.0,i am not a student and wanted to read a somewh...,0
4,1.0,this is a wonderfully written book i have lear...,0


In [5]:
stop_words = set(stopwords.words('english'))

# Assuming df1['reviewText'] is your column of text data
def remove_stopwords(text):
    # Tokenize the text into words
    word_tokens = word_tokenize(text)
    
    # Remove stopwords
    filtered_text = [word for word in word_tokens if word not in stop_words]
    
    # Reconstruct the text
    return ' '.join(filtered_text)

# Apply the function to remove stopwords from each review
df1['reviewText'] = df1['reviewText'].apply(lambda text: remove_stopwords(text))

In [6]:
def remove_alphanumeric_tokens(text):
    cleaned_text = re.sub(r'\b\w*\d+\w*\b', '', text)
    return re.sub(r'\s+', ' ', cleaned_text).strip()
df1['reviewText'] = df1['reviewText'].apply(remove_alphanumeric_tokens)


In [7]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df1['reviewText'])
word_index = tokenizer.word_index
vocab_size = len(word_index) +1

In [8]:
print(dict(list(word_index.items())[312780:312794]))
print(vocab_size)


{'uuuuffffff': 312781, 'jackandpam': 312782, 'trijicon': 312783, 'mro': 312784, 'expressway': 312785, 'gpses': 312786, 'imeadiatly': 312787, "'roomy": 312788, 'caplets': 312789, 'wildland': 312790, 'topomaps': 312791, 'maping': 312792, 'danbury': 312793}
312794


### sentence to word split

In [9]:
sequences = tokenizer.texts_to_sequences(df1['reviewText'])


In [10]:
len(df1['reviewText'])

299990

###  Padding sequences to the same length


In [11]:
max_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')
print(max_length)

2900


In [12]:
window_size=3

In [13]:
def generate_context_target_pairs(sequences, window_size):
   
    for seq in sequences:
        sequence_length = len(seq)
        for i, word_id in enumerate(seq):
            # Calculate window range
            start = max(i - window_size, 0)
            end = min(i + window_size + 1, sequence_length)
            
            # Prepare context words
            context = [seq[j] for j in range(start, end) if j != i]
            
            # Yield the current context and target word
            yield context, word_id

# Example of using the generator to print first few context-target pairs
for context, target in generate_context_target_pairs(sequences, window_size=3):
    print(f"Context: {context}, Target: {target}")
      # Remove or adjust this break statement as needed


In [ ]:
contexts = []
targets = []

for context, target in generate_context_target_pairs(sequences, window_size):
    contexts.append(context)
    targets.append(target)

In [ ]:
max_length = max(len(c) for c in contexts)  # Determine the maximum length
contexts_padded = pad_sequences(contexts, maxlen=max_length, padding='post', truncating='post')
targets = np.array(targets)
X_train, X_test, y_train, y_test = train_test_split(contexts_padded, targets, test_size=0.1)


In [ ]:
y_train

array([   12,    52,   582, ...,  5277, 22612,   797])

In [ ]:
print(y_train.shape)
X_train.shape

(19985219,)


(19985219, 6)

In [ ]:
embedding_dim=100
window_size = 3

In [ ]:
embedding_matrix = tf.Variable(tf.random.uniform([vocab_size, embedding_dim], -1.0, 1.0))


In [ ]:
def corruption(x, p=0.5):
    random_tensor = 1 - p
    random_tensor += tf.random.uniform(tf.shape(x))
    dropout_mask = tf.floor(random_tensor)
    return tf.divide(x, 1 - p) * dropout_mask

In [ ]:
class CDSAWEModel(Model):
    def __init__(self, vocab_size, embedding_dim):
        super(CDSAWEModel, self).__init__()
        self.embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim, embeddings_initializer='uniform')
        self.dense_sentiment = Dense(1, activation='sigmoid')
        
    def call(self, inputs):
        # inputs: list of two items, [context_words, target_word]
        context_words, target_word = inputs
        context_embeddings = self.embedding(context_words)
        
        # Average the context embeddings as per Equation (2)
        avg_context_embedding = tf.reduce_mean(context_embeddings, axis=1)
        
        # Corrupt the context embedding
        corrupted_context_embedding = corruption(avg_context_embedding)
        
        # Prediction from corrupted context
        sentiment_prediction = self.dense_sentiment(corrupted_context_embedding)
        
        return corrupted_context_embedding, sentiment_prediction

In [ ]:
model = CDSAWEModel(vocab_size, embedding_dim)

In [ ]:
context_words = tf.keras.Input(shape=(window_size,), dtype=tf.int32)
target_word = tf.keras.Input(shape=(), dtype=tf.int32)

In [ ]:
corrupted_context_embedding, sentiment_prediction = model([context_words, target_word])
